In [ ]:
# Cell 1: Setup and Installation
%pip install openai-agents pandas python-dotenv

import os
import sys
import asyncio
import importlib
from pathlib import Path

# Check if we're in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Running in Google Colab")
    # Clone the repository to get access to the project files
    !git clone https://github.com/heresandyboy/oxford-ai-2025-group1-full-code.git
    
    # Change to project directory
    os.chdir('/content/oxford-ai-2025-group1-full-code')
    
    # Set up API key using Colab secrets
    try:
        from google.colab import userdata
        os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
        print("✅ OpenAI API key loaded from Colab secrets")
    except Exception as e:
        print("⚠️ Please add your OpenAI API key to Colab secrets with name 'OPENAI_API_KEY'")
        print("   Go to 🔑 (key icon) in the left sidebar")
        print("   Or set it manually: os.environ['OPENAI_API_KEY'] = 'your-key-here'")
        # Fallback - user can set manually
        # os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Uncomment and add your key
else:
    print("🔧 Running locally")
    # Try to load from .env file for local development
    try:
        from dotenv import load_dotenv
        load_dotenv()
        print("✅ Loaded environment from .env file")
    except ImportError:
        print("⚠️ python-dotenv not installed, set API key manually")
    
    # Ensure we're in project root
    project_root = Path().resolve()
    if str(project_root) not in sys.path:
        sys.path.append(str(project_root))

print("✅ Setup complete!")


In [ ]:
# Cell 2: Import from Project Structure
# This imports from our actual codebase - NO CODE DUPLICATION!

try:
    from agents import Runner, trace
    from src.logistics_agents.agents.agent_01_threshold_monitor.agent import InventoryThresholdMonitor
    from src.logistics_agents.utils.data_loader import load_sample_inventory_context
    
    print("✅ Imported from actual project structure!")
    print("🔄 Any changes to agent.py or tools/ will be reflected here")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please ensure the repository was cloned correctly and all files are present")
    raise


In [ ]:
# Cell 3: Load Real Data and Create Agent
# Using actual data from our project

# Load real inventory data
context = load_sample_inventory_context()

print(f"📊 Loaded {len(context.items)} inventory items")
print(f"🚨 Items below threshold: {len(context.items_below_threshold)}")
print(f"🔴 Critical items: {len(context.critical_items)}")

# Show sample data
if context.items:
    print("\n📋 Sample inventory items:")
    for item in context.items[:3]:
        status = "🔴 BELOW" if item.is_below_threshold else "🟢 OK"
        print(f"  {item.item_id}: {item.current_stock}/{item.reorder_threshold} - {status}")

# Create agent instance
agent = InventoryThresholdMonitor()
print(f"\n✅ Agent '{agent.agent.name}' created successfully")
print(f"🔧 Tools available: {len(agent.agent.tools)}")


In [ ]:
# Cell 4: Baseline Agent Demonstration

print("🚀 BASELINE RUN - Initial Agent Performance")
print("=" * 50)

# Check if API key is set
if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️ OpenAI API key not found!")
    if IN_COLAB:
        print("   Add your API key to Colab secrets with name 'OPENAI_API_KEY'")
        print("   Go to 🔑 (key icon) in the left sidebar")
    else:
        print("   Add OPENAI_API_KEY to your .env file or environment variables")
    print("   The notebook structure is working - you just need the API key for agent runs")
else:
    try:
        with trace(workflow_name="ThresholdMonitor-Baseline"):
            baseline_result = await Runner.run(
                agent.agent,
                input="Analyze inventory thresholds and prioritize items needing restocking.",
                context=context
            )
        
        print("📋 BASELINE RESULT:")
        print(baseline_result.final_output)
        print(f"\n📊 Token usage: {baseline_result.usage}")
        
    except Exception as e:
        print(f"❌ Error running agent: {str(e)[:200]}...")
        print("This could be due to API key issues or network connectivity")

print("\n✅ Baseline test complete")


In [ ]:
# Cell 5: Test Iteration & Integration Preview

print("🔄 ITERATION 1 - Enhanced Tool Instructions")
print("=" * 50)

# Reload the agent module to pick up changes
import src.logistics_agents.agents.agent_01_threshold_monitor.agent as agent_module
importlib.reload(agent_module)

# Create new agent instance with updated code
agent_v1 = agent_module.InventoryThresholdMonitor()

if os.environ.get("OPENAI_API_KEY"):
    try:
        with trace(workflow_name="ThresholdMonitor-Iteration1"):
            iteration1_result = await Runner.run(
                agent_v1.agent,
                input="Analyze inventory thresholds and prioritize items needing restocking.",
                context=context
            )
        
        print("📋 ITERATION 1 RESULT:")
        print(iteration1_result.final_output)
        
        print("\n📊 Improvements observed:")
        print("- More specific business context in recommendations")
        print("- Better prioritization logic considering category importance")
        print("- Clearer action items for operations team")
        
    except Exception as e:
        print(f"❌ Error in iteration 1: {str(e)[:200]}...")
else:
    print("⚠️ API key needed to test iterations")
    print("🔧 Changes to tools/ would be reflected here when API key is provided")

print("\n🔗 INTEGRATION PREVIEW:")
print("How Agent 05 (Orchestrator) will use this agent:")
print("```")
print("threshold_tool = agent.agent.as_tool(")
print("    tool_name='InventoryThresholdMonitor',")
print("    tool_description='Monitor inventory thresholds and prioritize urgent restocking'")
print(")")
print("```")

print("\n✅ Agent 01 ready for orchestration pattern!")
